# Zero-Shot, Few-Shot, and Chain-of-Thought Prompting with Claude

This notebook demonstrates three fundamental prompting techniques using Anthropic's Claude API:

1. **Zero-Shot Prompting** - Ask the model to perform a task without any examples
2. **Few-Shot Prompting** - Provide a few examples to guide the model's responses
3. **Chain-of-Thought Prompting** - Ask the model to show its reasoning step by step

These techniques are essential for getting the best results from large language models.

## Step 1: Install Dependencies

In [1]:
%pip install anthropic python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Step 2: Set Up the Claude Client

In [2]:
import os
from dotenv import load_dotenv
import anthropic

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv("ANTHROPIC_API_KEY")

# Initialize the Anthropic client
client = anthropic.Anthropic(api_key=api_key)

# Choose your Claude model
# Options: claude-3-7-sonnet-20250219, claude-sonnet-4-20250514, claude-3-5-haiku-20241022
model = "claude-3-7-sonnet-20250219"

print(f"Using model: {model}")
print(f"API key configured: {'Yes' if api_key else 'No - please set ANTHROPIC_API_KEY'}")

Using model: claude-3-7-sonnet-20250219
API key configured: Yes


## Step 3: Create a Helper Function

In [3]:
def query_claude(prompt, temperature=0.7, max_tokens=300):
    """
    Send a prompt to Claude and return the response.
    
    Args:
        prompt: The user's prompt/question
        temperature: Controls randomness (0=focused, 1=creative)
        max_tokens: Maximum length of response
    
    Returns:
        The model's response text
    """
    response = client.messages.create(
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.content[0].text

---

## Technique 1: Zero-Shot Prompting

**Zero-shot prompting** means asking the model to perform a task without providing any examples. The model relies entirely on its pre-trained knowledge.

### When to use:
- Simple, well-defined tasks
- When the model likely understands the task from training
- Quick prototyping

### Example: General Knowledge Question

In [4]:
# Zero-Shot Prompting Example 1: General knowledge
zero_shot_prompt = "What are the benefits of regular exercise?"

zero_shot_response = query_claude(zero_shot_prompt)

print("ZERO-SHOT PROMPTING")
print("=" * 50)
print(f"Prompt: {zero_shot_prompt}")
print(f"\nResponse:\n{zero_shot_response}")

ZERO-SHOT PROMPTING
Prompt: What are the benefits of regular exercise?

Response:
# Benefits of Regular Exercise

Regular exercise offers numerous physical and mental health benefits:

**Physical Benefits:**
- Improved cardiovascular health and reduced risk of heart disease
- Weight management and increased metabolism
- Stronger muscles and bones, reducing risk of osteoporosis
- Better balance and flexibility, lowering injury risk
- Enhanced immune function
- Reduced risk of chronic diseases like type 2 diabetes and some cancers

**Mental Benefits:**
- Decreased stress, anxiety and depression symptoms
- Improved mood through endorphin release
- Better sleep quality
- Enhanced cognitive function and memory
- Increased energy levels and reduced fatigue

Even modest amounts of activity (150 minutes of moderate exercise weekly) can provide significant health benefits. The key is finding activities you enjoy so you can maintain consistency over time.


In [5]:
# Zero-Shot Prompting Example 2: Sentiment Analysis
zero_shot_sentiment = """Classify the sentiment of this review as positive, negative, or neutral:

"The restaurant had amazing food but the service was incredibly slow. I'm not sure if I'd go back."

Sentiment:"""

response = query_claude(zero_shot_sentiment, temperature=0)

print("ZERO-SHOT SENTIMENT ANALYSIS")
print("=" * 50)
print(f"Prompt: {zero_shot_sentiment}")
print(f"\nResponse: {response}")

ZERO-SHOT SENTIMENT ANALYSIS
Prompt: Classify the sentiment of this review as positive, negative, or neutral:

"The restaurant had amazing food but the service was incredibly slow. I'm not sure if I'd go back."

Sentiment:

Response: Sentiment: Mixed (or Neutral)

The review contains both positive elements ("amazing food") and negative elements ("service was incredibly slow"), with the reviewer expressing uncertainty about returning. This balanced mix of positive and negative aspects with an uncertain conclusion makes it a mixed or neutral sentiment overall.


---

## Technique 2: Few-Shot Prompting

**Few-shot prompting** provides the model with a few examples of the desired input-output pattern. This helps the model understand exactly what format and style you want.

### When to use:
- Tasks with specific output formats
- When zero-shot results aren't quite right
- Custom classification or formatting tasks

### Example: Translation with Consistent Style

In [6]:
# Few-Shot Prompting Example 1: Translation
few_shot_translation = """Translate the following English sentences to French:

1. Hello. -> Bonjour.
2. How are you? -> Comment ça va?
3. What is your name? ->"""

few_shot_response = query_claude(few_shot_translation)

print("FEW-SHOT PROMPTING: Translation")
print("=" * 50)
print(f"Prompt: {few_shot_translation}")
print(f"\nResponse: {few_shot_response}")

FEW-SHOT PROMPTING: Translation
Prompt: Translate the following English sentences to French:

1. Hello. -> Bonjour.
2. How are you? -> Comment ça va?
3. What is your name? ->

Response: 3. What is your name? -> Comment vous appelez-vous? (formal) or Comment t'appelles-tu? (informal)


In [7]:
# Few-Shot Prompting Example 2: Custom Classification
few_shot_classification = """Classify these customer support tickets into categories.

Ticket: "I can't log into my account"
Category: Account Access

Ticket: "When will my order arrive?"
Category: Shipping

Ticket: "I want a refund for my purchase"
Category: Returns/Refunds

Ticket: "The app keeps crashing on my phone"
Category:"""

response = query_claude(few_shot_classification, temperature=0)

print("FEW-SHOT PROMPTING: Classification")
print("=" * 50)
print(f"Prompt: {few_shot_classification}")
print(f"\nResponse: {response}")

FEW-SHOT PROMPTING: Classification
Prompt: Classify these customer support tickets into categories.

Ticket: "I can't log into my account"
Category: Account Access

Ticket: "When will my order arrive?"
Category: Shipping

Ticket: "I want a refund for my purchase"
Category: Returns/Refunds

Ticket: "The app keeps crashing on my phone"
Category:

Response: Ticket: "The app keeps crashing on my phone"
Category: Technical Issue

This ticket describes a software functionality problem with the mobile application, which falls under the Technical Issue category. The customer is reporting that the app is not working properly (repeatedly crashing), which is distinct from account access problems, shipping inquiries, or refund requests.


In [8]:
# Few-Shot Prompting Example 3: Structured Output
few_shot_structured = """Extract information from text and format as JSON.

Text: "John Smith is 35 years old and works as a software engineer."
JSON: {"name": "John Smith", "age": 35, "occupation": "software engineer"}

Text: "Maria Garcia, a 28-year-old doctor, lives in Boston."
JSON: {"name": "Maria Garcia", "age": 28, "occupation": "doctor", "city": "Boston"}

Text: "The CEO, Robert Chen, announced the merger at age 52."
JSON:"""

response = query_claude(few_shot_structured, temperature=0)

print("FEW-SHOT PROMPTING: Structured Output")
print("=" * 50)
print(f"Prompt: {few_shot_structured}")
print(f"\nResponse: {response}")

FEW-SHOT PROMPTING: Structured Output
Prompt: Extract information from text and format as JSON.

Text: "John Smith is 35 years old and works as a software engineer."
JSON: {"name": "John Smith", "age": 35, "occupation": "software engineer"}

Text: "Maria Garcia, a 28-year-old doctor, lives in Boston."
JSON: {"name": "Maria Garcia", "age": 28, "occupation": "doctor", "city": "Boston"}

Text: "The CEO, Robert Chen, announced the merger at age 52."
JSON:

Response: ```json
{
  "name": "Robert Chen",
  "age": 52,
  "occupation": "CEO",
  "event": "announced the merger"
}
```


---

## Technique 3: Chain-of-Thought Prompting

**Chain-of-Thought (CoT) prompting** asks the model to break down its reasoning into steps. This dramatically improves performance on complex reasoning tasks like math, logic, and multi-step problems.

### When to use:
- Math and arithmetic problems
- Logic puzzles
- Multi-step reasoning
- When you need to verify the model's reasoning

### Example: Math Problem

In [9]:
# Chain-of-Thought Example 1: Simple Math
cot_simple = """Calculate the following: If I have 3 apples and I buy 2 more, 
how many apples do I have in total? Please show your work step by step."""

cot_response = query_claude(cot_simple)

print("CHAIN-OF-THOUGHT: Simple Math")
print("=" * 50)
print(f"Prompt: {cot_simple}")
print(f"\nResponse:\n{cot_response}")

CHAIN-OF-THOUGHT: Simple Math
Prompt: Calculate the following: If I have 3 apples and I buy 2 more, 
how many apples do I have in total? Please show your work step by step.

Response:
# Calculating the Total Number of Apples

## Given:
- Initial number of apples: 3 apples
- Additional apples purchased: 2 apples

## Solution:
To find the total number of apples, I need to add the initial number of apples and the number of apples purchased.

Total apples = Initial apples + Purchased apples
Total apples = 3 + 2
Total apples = 5

## Answer:
If you have 3 apples and buy 2 more, you will have 5 apples in total.


In [ ]:
# Chain-of-Thought Example 2: Multi-step Word Problem
cot_complex = """Solve this problem step by step:

A store sells notebooks for $3 each and pens for $1.50 each. 
Sarah has $20 and wants to buy 4 notebooks. 
How many pens can she buy with the remaining money?

Think through this step by step:"""

response = query_claude(cot_complex, max_tokens=400)

print("CHAIN-OF-THOUGHT: Word Problem")
print("=" * 50)
print(f"Prompt: {cot_complex}")
print(f"\nResponse:\n{response}")

In [ ]:
# Chain-of-Thought Example 3: Logic Puzzle
cot_logic = """Solve this logic puzzle step by step:

Three friends - Alice, Bob, and Carol - each have a different pet: a cat, a dog, or a fish.
- Alice doesn't have the cat.
- Bob doesn't have the dog.
- Carol doesn't have the cat or the fish.

Who has which pet? Show your reasoning."""

response = query_claude(cot_logic, max_tokens=500)

print("CHAIN-OF-THOUGHT: Logic Puzzle")
print("=" * 50)
print(f"Prompt: {cot_logic}")
print(f"\nResponse:\n{response}")

---

## Comparison: Same Task, Different Techniques

Let's compare how the same task performs with different prompting techniques.

In [ ]:
# The task: Determine if a number is prime
number = 97

# Zero-shot
zero_shot = f"Is {number} a prime number? Answer yes or no."

# Few-shot
few_shot = f"""Determine if a number is prime:
Number: 7 -> Prime: Yes
Number: 12 -> Prime: No
Number: 23 -> Prime: Yes
Number: {number} -> Prime:"""

# Chain-of-thought
cot = f"""Is {number} a prime number? 
Think step by step: check if it's divisible by small prime numbers (2, 3, 5, 7, etc.).
Show your work and then give the final answer."""

print("COMPARISON: Is 97 prime?")
print("=" * 50)
print("\n--- Zero-Shot ---")
print(query_claude(zero_shot, temperature=0))

print("\n--- Few-Shot ---")
print(query_claude(few_shot, temperature=0))

print("\n--- Chain-of-Thought ---")
print(query_claude(cot, temperature=0, max_tokens=400))

---

## Summary

| Technique | Description | Best For |
|-----------|-------------|----------|
| **Zero-Shot** | No examples provided | Simple tasks, quick queries |
| **Few-Shot** | 2-5 examples provided | Custom formats, classification |
| **Chain-of-Thought** | Ask for step-by-step reasoning | Math, logic, complex problems |

### Tips for Better Prompting:

1. **Be specific** - Clearly state what you want
2. **Use examples** - When zero-shot doesn't work well
3. **Ask for reasoning** - Improves accuracy on complex tasks
4. **Set temperature** - Use 0 for factual tasks, higher for creative tasks
5. **Iterate** - Test and refine your prompts

### Key Differences from OpenAI:

| Feature | OpenAI | Claude |
|---------|--------|--------|
| Client | `openai.ChatCompletion.create()` | `client.messages.create()` |
| Model parameter | `engine` (Azure) | `model` |
| Response access | `response.choices[0].message['content']` | `response.content[0].text` |

## Exercise: Try Your Own Prompts

Use the cell below to experiment with different prompting techniques!

In [ ]:
# Your turn! Try different prompts here
my_prompt = """Your prompt here..."""

response = query_claude(my_prompt)
print(response)